In [ ]:
# Important : to make initial dataframe need to run RFMDash.ipynb and create TrainRFMRFMBL

%run "TrainInJupOnlineShopping.ipynb"


In [2]:
# Start Dash
# !pip install jupyter-dash
# !pip install dash-bootstrap-components
# pip install dash_mantine_components

import dash
from dash import Dash, dcc, dash_table, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import dash_mantine_components as dmc



# from mainOnlineShopping import TrainRFMRFMBL

In [3]:
## Start App
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [4]:
# app = dash.Dash(__name__)

# unique of user ids
# user_ids = TrainRFMRFMBL['user_id'].unique()

In [5]:
# data = TrainRFMRFMBL[['user_id', 'R', 'F', 'M', 'RNormScore', 'FNormScore', 'MNormScore']]

In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px

# فرض می‌کنیم TrainRFMRFMBL قبلاً تعریف شده است.
data = TrainRFMRFMBL[['user_id', 'R', 'F', 'M', 'RNormScore', 'FNormScore', 'MNormScore']]

# تبدیل داده به DataFrame
df = pd.DataFrame(data)

# ساخت اپلیکیشن Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    # نوار بالای داشبورد
    html.Div(
        children="مانیتورینگ رفتار مشتریان",
        style={
            'backgroundColor': 'lightblue',
            'padding': '10px',
            'fontSize': '20px',
            'textAlign': 'center',
            'fontWeight': 'bold'
        }
    ),

    # لای اوت اصلی با دو بخش: تب‌ها و محتویات
    html.Div([
        # نوار تب‌ها
        dcc.Tabs(id='tabs', value='tab1', children=[
            dcc.Tab(label='tab1', value='tab1'),
            dcc.Tab(label='tab2', value='tab2'),
            dcc.Tab(label='tab3', value='tab3'),
            dcc.Tab(label='tab4', value='tab4'),
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        # محتویات تب‌ها
        html.Div(id='tab-content', style={'width': '75%', 'display': 'inline-block'})
    ], style={'display': 'flex'})
])

# محتویات هر تب
@app.callback(
    Output('tab-content', 'children'),
    [Input('tabs', 'value'),
     State('user-id-dropdown', 'value')]
)
def render_tab(tab, selected_user_id):
    if tab == 'tab1':
        # فقط زمانی که به تب1 می‌رویم، dropdown ایجاد می‌شود
        dropdown = dcc.Dropdown(
            id='user-id-dropdown',
            options=[{'label': i, 'value': i} for i in df['user_id'].unique()],
            value=selected_user_id if selected_user_id else df['user_id'].iloc[0],  # مقدار اولیه
        )

        if selected_user_id:
            user_data = df[df['user_id'] == selected_user_id].iloc[0]
            # جدول برای R, F, M
            table = html.Table([
                html.Tr([html.Th("Feature"), html.Th("Value")]),
                html.Tr([html.Td("R"), html.Td(user_data['R'])]),
                html.Tr([html.Td("F"), html.Td(user_data['F'])]),
                html.Tr([html.Td("M"), html.Td(user_data['M'])]),
            ])
